# Pneumonia detection using chest x-ray ##

## Import useful librairies

In [1]:
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import warnings
warnings.filterwarnings('ignore')

## Load images ##

In [2]:
size_image = [224, 224]
train_path = 'chest_xray/chest_xray/TRAIN'
val_path = 'chest_xray/chest_xray/TEST'

## What is VGG16 model ? ##
A convolutional neural network is also known as a ConvNet, which is a kind of artificial neural network. A convolutional neural network has an input layer, an output layer, and various hidden layers. VGG16 is a type of CNN (Convolutional Neural Network) that is considered to be one of the best computer vision models to date. The creators of this model evaluated the networks and increased the depth using an architecture with very small (3 × 3) convolution filters, which showed a significant improvement on the prior-art configurations. They pushed the depth to 16–19 weight layers making it approx — 138 trainable parameters.

In [3]:
vgg = VGG16(input_shape = size_image + [3], weights = 'imagenet', include_top=False)

In [4]:
for i in vgg.layers:
    i.trainable = False

In [5]:
folder = glob('chest_xray/chest_xray/TRAIN/*')
x = Flatten()(vgg.output)

Here, we are using Dense because it's the task of predicting a label for each pixel of the image.

In [6]:
# first we define dense to predict each pixel
pred = Dense(len(folder), activation='softmax')(x)

# then we create the CNN model that we want to generate
model = Model(inputs = vgg.input, outputs = pred)

# finally we look at the summary of the model (architecture)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

When our model is created, we can compile it :
- Categorical cross-entropy is used as a loss function for multi-class classification model where there are two or more output labels. The output label is assigned one-hot category encoding value in form of 0s and 1. The output label, if present in integer form, is converted into categorical encoding using keras.utils to_categorical method.
- Adam, derived from Adaptive Moment Estimation, is an optimization algorithm. The Adam optimizer makes use of a combination of ideas from other optimizers. Similar to the momentum optimizer, Adam makes use of an exponentially decaying average of past gradients.

In [7]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

## Data generator ##
Keras ImageDataGenerator is used for getting the input of the original data and further, it makes the transformation of this data on a random basis and gives the output resultant containing only the data that is newly transformed. It does not add the data. Keras image data generator class is also used to carry out data augmentation where we aim to gain the overall increment in the generalization of the model. Operations such as rotations, translations, shearin, scale changes, and horizontal flips are carried out randomly in data augmentation using an image data generator.

In [8]:
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2,horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory('chest_xray/chest_xray/TRAIN',target_size = (224, 224),batch_size = 10,class_mode = 'categorical')
test_set = test_datagen.flow_from_directory('chest_xray/chest_xray/TEST',target_size = (224, 224),batch_size = 10,class_mode = 'categorical')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


## Fit the model ##
This step is very long because it has to fit our model on each data, it means on each picture of our folder chest_xray.

In [9]:
model_fit = model.fit_generator(training_set,validation_data = test_set,epochs = 1,steps_per_epoch = len(training_set),validation_steps = len(test_set))

522/522 [==============================] - 1267s 2s/step - loss: 0.1996 - accuracy: 0.9300 - val_loss: 0.3292 - val_accuracy: 0.9135


## Save the model ##

In [10]:
model.save('model_test.h5')